# Framework overview

Pymob is built around {class}`pymob.simulation.Simulation`, which is the object where all necessary information are pooled. For configuration, `pymob` relies on {class}`pymob.sim.config.Config`, which uses [`pydantic`](https://docs.pydantic.dev/latest/) to validate the configuration, before it is used to set up the simulation. 

In [ ]:
from pymob import SimulationBase

# initializing a Simulation with a config file
sim = SimulationBase(config="case_studies/quickstart/scenarios/test/settings.cfg")

# accessing the config file
sim.config

Config(case_study=Casestudy(init_root='/home/flo-schu/projects/pymob/docs/source/user_guide', root='.', name='quickstart', scenario='test', package='case_studies', modules=['sim', 'mod', 'prob', 'data', 'plot'], simulation='Simulation', output=None, data=None, observations='observations.nc', logging='DEBUG', logfile=None, output_path='case_studies/quickstart/results/test', data_path='case_studies/quickstart/data', default_settings_path='case_studies/quickstart/scenarios/test/settings.cfg'), simulation=Simulation(model=None, solver=None, y0=[], x_in=[], input_files=[], n_ode_states=1, batch_dimension='batch_id', x_dimension='time', modeltype='deterministic', solver_post_processing=None, seed=1), data_structure=Datastructure(y=DataVariable(dimensions=['x'], min=-5.690912333645177, max=5.891166954282328, observed=True, dimensions_evaluator=None)), solverbase=Solverbase(x_dim='time', exclude_kwargs_model=('t', 'time', 'x_in', 'y', 'x', 'Y', 'X'), exclude_kwargs_postprocessing=('t', 'time',

## Pymob API

![framework-overview](./figures/pymob_overview.png)

### Pymob exposes the following input and output interfaces

#### Config

Pymob uses [`pydantic`](https://docs.pydantic.dev/latest/) Models for validation of the configuration files. The configuration is organized into sections, e.g.

In [ ]:
sim.config.data_structure

Datastructure(y=DataVariable(dimensions=['x'], min=-5.690912333645177, max=5.891166954282328, observed=True, dimensions_evaluator=None))

Configurations can be changed in the files before a simulation is initialized from a config file, or directly in the script.

In [19]:
sim.config.data_structure.y.min = 0
print(sim.config.data_structure.y)

dimensions=['x'] min=0.0 max=5.891166954282328 observed=True dimensions_evaluator=None


As can be seen in the figure above, it is the communication between Simulation class and config files is bidirectional, this means, Simulations can be created from config files or in a scripting environment, and successively exported to config files. For more information see [configuration](case_studies.md#configuration) for details

#### Solver

Solvers solve the model. In order to automatize dimension handling and solving the model for the correct coordinates. Solvers subclass {class}`pymob.solver.SolverBase`. 

In [17]:
sim.solver

#### Model

Models are provided as plain Python functions. 

In [18]:
sim.model

#### Observations

Observations are required to be xarray Datasets. An [`xarray.Dataset`](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html) is a collection of annotated arrays, using HDF5 data formats for input/output operations.

#### Simulation results

Simulation results are returned by the solver. Plainly they are returned as dictionaries containing NDarrays. However, due to the information contained in the observations dataset, the results dictionary is automatically casted to an [`xarray.Dataset`](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html), which has the same shape as the observations. This makes comparisons between observations and simulations extremely easy.

#### Parameter estimates

Parameter estimates are harmonized by reporting them as [`arviz.InferenceData`](https://python.arviz.org/en/latest/getting_started/WorkingWithInferenceData.html) using `xarray.Datasets` under the hood. Thereby `pymob` supports variably dimensional datasets


## Parameter estimation

Parameter estimation is implemented through backends, which can be seen as converters between the {class}`pymob.simulation.SimulationBase` object and the API of the Inference tool. Inference backends are selected by using 

In [7]:
sim.set_inferer("numpyro")

Jax 64 bit mode: False
Absolute tolerance: 1e-07


### Supported Algorithms and Planned Features

| Backend | Supported Algorithms | Inference | Hierarchical Models |
| :--- | --- | --- | --- |
| `numpyro` | Markov Chain Monte Carlo (MCMC), Stochastic Variational Inference (SVI) | ✅ | ✅ |
| `pymoo` | (Global) Multi-objective optimization | ✅ | plan |
| `pyabc` | Approximate Bayes | ✅ | plan |
| `scipy` | Local optimization (`minimize`) | dev | plan |
| `pymc` | MCMC | plan | plan |
| `sbi` | Simulation Based Inference (in planning) | hold | hold |
| `interactive ` | interactive backend in jupyter notebookswith parameter sliders | ✅ | plan |